# `recent_combined_name`/`route_id` conundrum
**6/19/2025 Notes**
* City of Santa Maria looks the same with and without the pipes, but I can see multiple route_ids go with multiple `recent_combined_name`. Take a closer look
* Samething with Yolobus

In [53]:
import pandas as pd
import yaml
from segment_speed_utils import time_series_utils
from shared_utils import portfolio_utils, rt_dates
from update_vars import GTFS_DATA_DICT, SCHED_GCS

with open("../_shared_utils/shared_utils/portfolio_organization_name.yml", "r") as f:
    PORTFOLIO_ORGANIZATIONS_DICT = yaml.safe_load(f)

In [2]:
import geopandas as gpd
import google.auth
from update_vars import GTFS_DATA_DICT, RT_SCHED_GCS, SCHED_GCS, SEGMENT_GCS

credentials, project = google.auth.default()

import gcsfs
from calitp_data_analysis import geography_utils, utils
from shared_utils import gtfs_utils_v2, portfolio_utils, publish_utils

In [64]:
CLEANED_ROUTE_NAMING = GTFS_DATA_DICT.schedule_tables.route_identification

df = pd.read_parquet(f"{SCHED_GCS}{CLEANED_ROUTE_NAMING}.parquet").pipe(
    portfolio_utils.standardize_portfolio_organization_names,
    PORTFOLIO_ORGANIZATIONS_DICT,
)

In [4]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

## Find other routes in which their `route_short_name` , `route_long_name`, and `route_id` are different but they share the same `recent_combined_name`
* [Link](https://github.com/cal-itp/data-analyses/issues/1527#issuecomment-2982251267)
* It's ok for recent_combined_name to group multiple route_ids only when route_short_name and route_long_name are the same, see the LA Metro route_id example.
* It is not ok to group them when route_short_name and route_long_name are different, and the time_series_utils operators that get extra parsing only had that to handle cases like:
* https://github.com/cal-itp/data-analyses/blob/9505ad4445de05119b78d4ad35ac89c23ed8a7dc/rt_segment_speeds/segment_speed_utils/time_series_utils.py#L84-L105

In [5]:
public_feeds = gtfs_utils_v2.filter_to_public_schedule_gtfs_dataset_keys()

In [65]:
df = df.pipe(
    publish_utils.exclude_private_datasets, public_gtfs_dataset_keys=public_feeds
)

In [154]:
analysis_date_list = rt_dates.y2025_dates
FILE = GTFS_DATA_DICT.schedule_tables.gtfs_key_crosswalk
crosswalk_cols = [
    "schedule_gtfs_dataset_key",
    "name",
    "organization_name",
]

crosswalk_df = time_series_utils.concatenate_datasets_across_dates(
    SCHED_GCS, FILE, analysis_date_list, data_type="df", columns=crosswalk_cols
).drop_duplicates()

### City of Santa Maria `Santa Maria Schedule`

In [7]:
santa_maria = df.loc[
    (df.portfolio_organization_name.str.contains("City of Santa Maria"))
    & (df.recent_combined_name == "13X__13X Transit Center/PVHS/N. Broadway")
]

In [155]:
crosswalk_df.loc[
    crosswalk_df.organization_name.str.contains(
        "Santa Maria",
    )
]

,schedule_gtfs_dataset_key,name,organization_name,service_date
211,73105f2d1cabc8170ab066d96863c5d5,Santa Maria Schedule,City of Santa Maria,2025-01-15
211,73105f2d1cabc8170ab066d96863c5d5,Santa Maria Schedule,City of Santa Maria,2025-02-12
210,73105f2d1cabc8170ab066d96863c5d5,Santa Maria Schedule,City of Santa Maria,2025-03-12
217,73105f2d1cabc8170ab066d96863c5d5,Santa Maria Schedule,City of Santa Maria,2025-04-16
216,73105f2d1cabc8170ab066d96863c5d5,Santa Maria Schedule,City of Santa Maria,2025-05-14


In [8]:
len(santa_maria)

35

In [9]:
santa_maria[
    [
        "service_date",
        "recent_combined_name",
        "portfolio_organization_name",
        "schedule_gtfs_dataset_key",
        "route_long_name",
        "route_short_name",
        "route_id",
    ]
].sort_values(by=["service_date"])

,service_date,recent_combined_name,portfolio_organization_name,schedule_gtfs_dataset_key,route_long_name,route_short_name,route_id
23616,2023-07-12,13X__13X Transit Center/PVHS/N. Broadway,City of Santa Maria,73105f2d1cabc8170ab066d96863c5d5,13X Transit Center/PVHS/N. Broadway,13X,5b894962-da2e-442f-b9ec-df43cd2f80d0
26129,2023-08-15,13X__13X Transit Center/PVHS/N. Broadway,City of Santa Maria,73105f2d1cabc8170ab066d96863c5d5,13X Transit Center/PVHS/N. Broadway,13X,5b894962-da2e-442f-b9ec-df43cd2f80d0
28692,2023-09-13,13X__13X Transit Center/PVHS/N. Broadway,City of Santa Maria,73105f2d1cabc8170ab066d96863c5d5,13X Transit Center/PVHS/N. Broadway,13X,5b894962-da2e-442f-b9ec-df43cd2f80d0
31230,2023-10-09,13X__13X Transit Center/PVHS/N. Broadway,City of Santa Maria,73105f2d1cabc8170ab066d96863c5d5,13X Transit Center/PVHS/N. Broadway,13X,5b894962-da2e-442f-b9ec-df43cd2f80d0
33977,2023-10-10,13X__13X Transit Center/PVHS/N. Broadway,City of Santa Maria,73105f2d1cabc8170ab066d96863c5d5,13X Transit Center/PVHS/N. Broadway,13X,5b894962-da2e-442f-b9ec-df43cd2f80d0
36466,2023-10-11,13X__13X Transit Center/PVHS/N. Broadway,City of Santa Maria,73105f2d1cabc8170ab066d96863c5d5,13X Transit Center/PVHS/N. Broadway,13X,5b894962-da2e-442f-b9ec-df43cd2f80d0
39031,2023-10-12,13X__13X Transit Center/PVHS/N. Broadway,City of Santa Maria,73105f2d1cabc8170ab066d96863c5d5,13X Transit Center/PVHS/N. Broadway,13X,5b894962-da2e-442f-b9ec-df43cd2f80d0
41619,2023-10-13,13X__13X Transit Center/PVHS/N. Broadway,City of Santa Maria,73105f2d1cabc8170ab066d96863c5d5,13X Transit Center/PVHS/N. Broadway,13X,5b894962-da2e-442f-b9ec-df43cd2f80d0
47378,2023-11-15,13X__13X Transit Center/PVHS/N. Broadway,City of Santa Maria,73105f2d1cabc8170ab066d96863c5d5,13X Transit Center/PVHS/N. Broadway,13X,5b894962-da2e-442f-b9ec-df43cd2f80d0
49943,2023-12-13,13X__13X Transit Center/PVHS/N. Broadway,City of Santa Maria,73105f2d1cabc8170ab066d96863c5d5,13X Transit Center/PVHS/N. Broadway,13X,5b894962-da2e-442f-b9ec-df43cd2f80d0


In [10]:
santa_maria2 = df.loc[
    (df.portfolio_organization_name.str.contains("City of Santa Maria"))
    & (df.recent_combined_name.str.contains("Orcutt Express"))
]

In [11]:
santa_maria2[
    [
        "service_date",
        "recent_combined_name",
        "portfolio_organization_name",
        "route_long_name",
        "route_short_name",
        "route_id",
        "schedule_gtfs_dataset_key",
    ]
].sort_values(by=["service_date"]).tail(10)

,service_date,recent_combined_name,portfolio_organization_name,route_long_name,route_short_name,route_id,schedule_gtfs_dataset_key
92043,2024-10-16,12X__12X Broadway/Orcutt Express,City of Santa Maria,12X Broadway/Orcutt Express,12X,12X,73105f2d1cabc8170ab066d96863c5d5
94647,2024-10-17,12X__12X Broadway/Orcutt Express,City of Santa Maria,12X Broadway/Orcutt Express,12X,12X,73105f2d1cabc8170ab066d96863c5d5
97367,2024-10-18,12X__12X Broadway/Orcutt Express,City of Santa Maria,12X Broadway/Orcutt Express,12X,12X,73105f2d1cabc8170ab066d96863c5d5
103069,2024-11-13,12X__12X Broadway/Orcutt Express,City of Santa Maria,12X Broadway/Orcutt Express,12X,12X,73105f2d1cabc8170ab066d96863c5d5
105854,2024-12-11,12X__12X Broadway/Orcutt Express,City of Santa Maria,12X Broadway/Orcutt Express,12X,12X,73105f2d1cabc8170ab066d96863c5d5
108476,2025-01-15,12X__12X Broadway/Orcutt Express,City of Santa Maria,12X Broadway/Orcutt Express,12X,12X,73105f2d1cabc8170ab066d96863c5d5
111088,2025-02-12,12X__12X Broadway/Orcutt Express,City of Santa Maria,12X Broadway/Orcutt Express,12X,12X,73105f2d1cabc8170ab066d96863c5d5
113612,2025-03-12,12X__12X Broadway/Orcutt Express,City of Santa Maria,12X Broadway/Orcutt Express,12X,12X,73105f2d1cabc8170ab066d96863c5d5
116235,2025-04-16,12X__12X Broadway/Orcutt Express,City of Santa Maria,12X Broadway/Orcutt Express,12X,12X,73105f2d1cabc8170ab066d96863c5d5
118652,2025-05-14,12X__12X Broadway/Orcutt Express,City of Santa Maria,12X Broadway/Orcutt Express,12X,12X,73105f2d1cabc8170ab066d96863c5d5


### Yolo Bus

In [12]:
yolo1 = df.loc[
    (df.portfolio_organization_name.str.contains("Yolo County Transportation District"))
    & (df.recent_combined_name.str.contains("Causeway"))
]

In [13]:
yolo1[
    [
        "service_date",
        "recent_combined_name",
        "portfolio_organization_name",
        "route_long_name",
        "route_short_name",
        "route_id",
        "schedule_gtfs_dataset_key",
    ]
].sort_values(by=["service_date"])

,service_date,recent_combined_name,portfolio_organization_name,route_long_name,route_short_name,route_id,schedule_gtfs_dataset_key
429,2023-03-15,138__Causeway Connection,Yolo County Transportation District,Causeway Connection,138,ffc3845f-a0cf-4553-81c1-f53d0470f0f3,97b8c635bbc4cdd02bc5c27d0e83df3c
2732,2023-04-10,138__Causeway Connection,Yolo County Transportation District,Causeway Connection,138,ffc3845f-a0cf-4553-81c1-f53d0470f0f3,97b8c635bbc4cdd02bc5c27d0e83df3c
5089,2023-04-11,138__Causeway Connection,Yolo County Transportation District,Causeway Connection,138,ffc3845f-a0cf-4553-81c1-f53d0470f0f3,97b8c635bbc4cdd02bc5c27d0e83df3c
7444,2023-04-12,138__Causeway Connection,Yolo County Transportation District,Causeway Connection,138,ffc3845f-a0cf-4553-81c1-f53d0470f0f3,97b8c635bbc4cdd02bc5c27d0e83df3c
9804,2023-04-13,138__Causeway Connection,Yolo County Transportation District,Causeway Connection,138,ffc3845f-a0cf-4553-81c1-f53d0470f0f3,97b8c635bbc4cdd02bc5c27d0e83df3c
12181,2023-04-14,138__Causeway Connection,Yolo County Transportation District,Causeway Connection,138,ffc3845f-a0cf-4553-81c1-f53d0470f0f3,97b8c635bbc4cdd02bc5c27d0e83df3c
17114,2023-05-17,138__Causeway Connection,Yolo County Transportation District,Causeway Connection,138,b6bc2958-333f-4d1e-8579-b75b2698b6b4,97b8c635bbc4cdd02bc5c27d0e83df3c
19580,2023-06-14,138__Causeway Connection,Yolo County Transportation District,Causeway Connection,138,b6bc2958-333f-4d1e-8579-b75b2698b6b4,97b8c635bbc4cdd02bc5c27d0e83df3c
21997,2023-07-12,138__Causeway Connection,Yolo County Transportation District,Causeway Connection,138,b6bc2958-333f-4d1e-8579-b75b2698b6b4,97b8c635bbc4cdd02bc5c27d0e83df3c
24384,2023-08-15,138__Causeway Connection,Yolo County Transportation District,Causeway Connection,138,b6bc2958-333f-4d1e-8579-b75b2698b6b4,97b8c635bbc4cdd02bc5c27d0e83df3c


In [156]:
crosswalk_df.loc[
    crosswalk_df.organization_name.str.contains(
        "Yolo",
    )
]

,schedule_gtfs_dataset_key,name,organization_name,service_date
51,3c62ad6ee589d56eca915ce291a5df0a,Yolobus Schedule,Yolo County Transportation District,2025-01-15
52,3c62ad6ee589d56eca915ce291a5df0a,Yolobus Schedule,Yolo County Transportation District,2025-02-12
53,3c62ad6ee589d56eca915ce291a5df0a,Yolobus Schedule,Yolo County Transportation District,2025-03-12
51,24956c9535f71f462472e2a1ceb5567c,Yolobus Schedule,Yolo County Transportation District,2025-04-16
49,7535a7b8c19aeaef607abf014ee2fd3f,Yolobus Schedule,Yolo County Transportation District,2025-05-14


### Elk Grove - should genuinely remove this.

In [14]:
elkgrove1 = df.loc[
    (df.portfolio_organization_name.str.contains("City of Elk Grove"))
    & (df.recent_combined_name.str.contains("Commuter"))
]

#### Called Elk Grove Commuter Route

In [15]:
elkgrove1[
    [
        "service_date",
        "recent_combined_name",
        "portfolio_organization_name",
        "route_long_name",
        "route_short_name",
        "route_id",
        "schedule_gtfs_dataset_key",
    ]
].sort_values(by=["service_date"]).tail(10)

,service_date,recent_combined_name,portfolio_organization_name,route_long_name,route_short_name,route_id,schedule_gtfs_dataset_key
118289,2025-05-14,E10__Elk Grove Commuter Route,City of Elk Grove,Elk Grove Commuter Route,E10,E10,89b0a03002512a909fe8e1a5d00a78a5
118290,2025-05-14,E11__Elk Grove Commuter Route,City of Elk Grove,Elk Grove Commuter Route,E11,E11,89b0a03002512a909fe8e1a5d00a78a5
118298,2025-05-14,E12__Elk Grove Commuter Route,City of Elk Grove,Elk Grove Commuter Route,E12,E12,89b0a03002512a909fe8e1a5d00a78a5
118299,2025-05-14,E13__Elk Grove Commuter Route,City of Elk Grove,Elk Grove Commuter Route,E13,E13,89b0a03002512a909fe8e1a5d00a78a5
118300,2025-05-14,E14__Elk Grove Commuter Route,City of Elk Grove,Elk Grove Commuter Route,E14,E14,89b0a03002512a909fe8e1a5d00a78a5
118301,2025-05-14,E15__Elk Grove Commuter Route,City of Elk Grove,Elk Grove Commuter Route,E15,E15,89b0a03002512a909fe8e1a5d00a78a5
118302,2025-05-14,E16__Elk Grove Commuter Route,City of Elk Grove,Elk Grove Commuter Route,E16,E16,89b0a03002512a909fe8e1a5d00a78a5
118303,2025-05-14,E17__Elk Grove Commuter Route,City of Elk Grove,Elk Grove Commuter Route,E17,E17,89b0a03002512a909fe8e1a5d00a78a5
118304,2025-05-14,E19__Elk Grove Commuter Route,City of Elk Grove,Elk Grove Commuter Route,E19,E19,89b0a03002512a909fe8e1a5d00a78a5
118706,2025-05-14,E18__Elk Grove Commuter Route,City of Elk Grove,Elk Grove Commuter Route,E18,E18,89b0a03002512a909fe8e1a5d00a78a5


In [16]:
elkgrove2 = df.loc[
    (df.portfolio_organization_name.str.contains("City of Elk Grove"))
    & (df.recent_combined_name.str.contains("Local"))
]

In [17]:
elkgrove2[
    [
        "service_date",
        "recent_combined_name",
        "portfolio_organization_name",
        "route_long_name",
        "route_short_name",
        "route_id",
        "schedule_gtfs_dataset_key",
    ]
].sort_values(by=["service_date"]).tail(10)

,service_date,recent_combined_name,portfolio_organization_name,route_long_name,route_short_name,route_id,schedule_gtfs_dataset_key
115880,2025-04-16,E112__Elk Grove Local Route,City of Elk Grove,Elk Grove Local Route,E112,E112,89b0a03002512a909fe8e1a5d00a78a5
115879,2025-04-16,E111__Elk Grove Local Route,City of Elk Grove,Elk Grove Local Route,E111,E111,89b0a03002512a909fe8e1a5d00a78a5
115878,2025-04-16,E110__Elk Grove Local Route,City of Elk Grove,Elk Grove Local Route,E110,E110,89b0a03002512a909fe8e1a5d00a78a5
118291,2025-05-14,E110__Elk Grove Local Route,City of Elk Grove,Elk Grove Local Route,E110,E110,89b0a03002512a909fe8e1a5d00a78a5
118292,2025-05-14,E111__Elk Grove Local Route,City of Elk Grove,Elk Grove Local Route,E111,E111,89b0a03002512a909fe8e1a5d00a78a5
118293,2025-05-14,E112__Elk Grove Local Route,City of Elk Grove,Elk Grove Local Route,E112,E112,89b0a03002512a909fe8e1a5d00a78a5
118294,2025-05-14,E113__Elk Grove Local Route,City of Elk Grove,Elk Grove Local Route,E113,E113,89b0a03002512a909fe8e1a5d00a78a5
118295,2025-05-14,E114__Elk Grove Local Route,City of Elk Grove,Elk Grove Local Route,E114,E114,89b0a03002512a909fe8e1a5d00a78a5
118296,2025-05-14,E115__Elk Grove Local Route,City of Elk Grove,Elk Grove Local Route,E115,E115,89b0a03002512a909fe8e1a5d00a78a5
118297,2025-05-14,E116__Elk Grove Local Route,City of Elk Grove,Elk Grove Local Route,E116,E116,89b0a03002512a909fe8e1a5d00a78a5


In [157]:
crosswalk_df.loc[
    crosswalk_df.organization_name.str.contains(
        "Elk Grove",
    )
]

,schedule_gtfs_dataset_key,name,organization_name,service_date
177,89b0a03002512a909fe8e1a5d00a78a5,Elk Grove Schedule,City of Elk Grove,2025-03-12
180,89b0a03002512a909fe8e1a5d00a78a5,Elk Grove Schedule,City of Elk Grove,2025-04-16
179,89b0a03002512a909fe8e1a5d00a78a5,Elk Grove Schedule,City of Elk Grove,2025-05-14


##  Caltrain we don't publish, so we don't have to look through it.

## Find other routes like VCTC's Coastal Express

In [18]:
with_pipe = df.pipe(time_series_utils.clean_standardized_route_names).astype(str)

In [21]:
agg1 = (
    with_pipe.groupby(
        [
            "service_date",
            "schedule_gtfs_dataset_key",
            "portfolio_organization_name",
            "recent_combined_name",
        ]
    )
    .agg(
        {
            "route_id": "nunique",
            "route_long_name": "nunique",
            "route_short_name": "nunique",
        }
    )
    .reset_index()
)

In [23]:
# Find the routes with more than one for route_id, route_long_name, and route_short_name
agg1_filtered = agg1.loc[(agg1.route_id > 1) & (agg1.route_short_name > 1)]

In [24]:
agg1_filtered.shape

(576, 7)

In [25]:
agg1_filtered.sample(3)

,service_date,schedule_gtfs_dataset_key,portfolio_organization_name,recent_combined_name,route_id,route_long_name,route_short_name
27498,2023-09-13,bd24573e82aa13be83eb7143179dd099,Mountain View Transportation Management Association,"A - AM__Whisman, Clyde & Middlefield",2,2,2
105382,2025-01-15,6869ab0f62d858fcd5d070e651609224,Mountain View Transportation Management Association,"B - PM__Shoreline, Crittenden, La Avenida",2,2,2
102791,2024-12-11,6869ab0f62d858fcd5d070e651609224,Mountain View Transportation Management Association,"B - PM__Shoreline, Crittenden, La Avenida",2,2,2


### Troubleshoot the operators below?
* Roseville: Same `recent_combined_name` has two different `route_id`?
* VCTC: done, commented them out of `time_series_utils`
* SF Bay Area Rapid Transit District: we don't publish any rt data
* Mountain View
* LA Metro: we already know 
* City of Santa Maria: Situation in which the `recent_combined_name` should be the same -> the opposite of VCTC's problem of 80-89
* Peninsula Corridor: Only one route and the `recent_combined_name` is blank so maybe it's just a glitch?
* City of Elk Grove: done, commented them out of `time_series_utils`

In [26]:
routes_to_focus = agg1_filtered.sort_values(
    by=["portfolio_organization_name", "service_date", "recent_combined_name"],
    ascending=[True, False, True],
)[
    ["portfolio_organization_name", "service_date", "recent_combined_name"]
].drop_duplicates(
    subset=["portfolio_organization_name", "recent_combined_name"]
)

In [27]:
routes_to_focus.loc[
    routes_to_focus.service_date >= "2024-12-11",
].portfolio_organization_name.unique()

array(['City of Roseville', 'City of Santa Maria',
       'Los Angeles County Metropolitan Transportation Authority',
       'Mountain View Transportation Management Association',
       'San Francisco Bay Area Rapid Transit District'], dtype=object)

In [28]:
routes_to_focus.loc[routes_to_focus.service_date >= "2024-12-11",]

,portfolio_organization_name,service_date,recent_combined_name
115479,City of Roseville,2025-05-14,10_PM__PM COMMUTER 10
115480,City of Roseville,2025-05-14,1_AM__AM COMMUTER 1
115481,City of Roseville,2025-05-14,2_AM__AM COMMUTER 2
115482,City of Roseville,2025-05-14,3_AM__AM COMMUTER 3
115483,City of Roseville,2025-05-14,4_PM__PM COMMUTER 4
115484,City of Roseville,2025-05-14,5_AM__AM COMMUTER 5
115485,City of Roseville,2025-05-14,6_AM__AM COMMUTER 6
115486,City of Roseville,2025-05-14,7_AM__AM COMMUTER 7
115487,City of Roseville,2025-05-14,9_AM__AM COMMUTER 9
115890,City of Santa Maria,2025-05-14,Rt 1. Transit Ctr to Preisker Park Via N. Broadway


In [29]:
agg1_filtered.portfolio_organization_name.unique()

array(['City of Roseville',
       'San Francisco Bay Area Rapid Transit District',
       'Mountain View Transportation Management Association',
       'Los Angeles County Metropolitan Transportation Authority',
       'Amtrak', 'City of Santa Maria'], dtype=object)

In [129]:
def inspect_recent_rows(df: pd.DataFrame, route_long_name: str):
    df2 = (
        df.loc[df.route_long_name == route_long_name][
            [
                "service_date",
                "recent_combined_name",
                "portfolio_organization_name",
                "schedule_gtfs_dataset_key",
                "route_long_name",
                "route_short_name",
                "route_id",
            ]
        ]
        .sort_values(by=["service_date"])
        .drop_duplicates()
        .tail(10)
    )
    display(df2)
    return

In [31]:
def process_transit_routes() -> gpd.GeoDataFrame:
    """
    Select the most recent transit route.
    Also count how many routes there are for each operator.
    """
    OPERATOR_ROUTE = GTFS_DATA_DICT.digest_tables.operator_routes_map

    op_geography_df = gpd.read_parquet(
        f"{RT_SCHED_GCS}{OPERATOR_ROUTE}.parquet",
        storage_options={"token": credentials.token},
    )

    most_recent_routes = publish_utils.filter_to_recent_date(
        df=op_geography_df,
        group_cols=["portfolio_organization_name", "route_id"],
    )

    # Calc length of route
    most_recent_routes = most_recent_routes.assign(
        route_length_feet=most_recent_routes.geometry.to_crs(
            geography_utils.CA_NAD83Albers_ft
        ).length
    )

    # Drop duplicates
    most_recent_routes = most_recent_routes.drop_duplicates(
        subset=[
            "portfolio_organization_name",
            "recent_combined_name",
            "service_date",
            "route_id",
        ]
    )
    return most_recent_routes

In [32]:
routes = process_transit_routes()

In [33]:
routes.loc[
    routes.portfolio_organization_name == "Peninsula Corridor Joint Powers Board"
].recent_combined_name.unique()

array(['Bullet', 'Local', 'LTD 3', 'LTD 4', 'LTD 5',
       '8XX peak hour connector from/to Gilroy', '',
       'South Santa Clara County Connector'], dtype=object)

In [34]:
routes.loc[
    routes.portfolio_organization_name
    == "Mountain View Transportation Management Association"
].recent_combined_name.unique()

array(['A - PM Middlefield, Clyde & Whisman',
       'B - PM Shoreline, Crittenden, La Avenida',
       'C - AM Charleston, Garcia & San Antonio (Counterclockwise loop)',
       'D - AM San Antonio, Garcia & Charleston (Clockwise loop)'],
      dtype=object)

In [35]:
routes.loc[
    routes.portfolio_organization_name == "City of Santa Maria",
].recent_combined_name.unique()

array(['Rt 7. A. H. College, Crossroads Shopping Center via Boone St, Bradley Rd.',
       'Rt 4. Transit Center to SMH school to VTC via Cook St., Thornburg., Betteravia Rd.',
       'Route 30 - Santa Maria Transit Center/Vandenberg/Lompoc',
       '13X Transit Center/PVHS/N. Broadway',
       'Rt 1. Transit Ctr to Preisker Park Via N. Broadway',
       'Rt 6. Oak Knolls to Old Orcutt-East to West-Outbound',
       'Route 20 - Santa Maria Transit Center/Los Alamos/Buellton/Solvang-OB',
       '12X Broadway/Orcutt Express',
       'Rt 3. Transit Center to Marian Hospital to PVH School, via E. Main, Suey Ln.',
       "Rt 5. Transit Center to Gov't Cntr to Evergreen Shopping Center via Miller St., S.M. Way",
       'Rt 2. Transit Center to PVH School via Western., Donovan Rd',
       'Rt 8. Tanglewood to Crossroads Shopping Center via McCoy Ln.',
       'Rt 9. Transit Center to PVH via Alvin Ave.', 'Mall Shuttle',
       "R11. Transit Center to Gov't Center via S. Broadway",
       "Rt 1

In [36]:
routes.loc[
    routes.portfolio_organization_name == "City of Roseville",
].recent_combined_name.unique()

array(['10_PM PM COMMUTER 10', '1_AM AM COMMUTER 1', '2_AM AM COMMUTER 2',
       '3_AM AM COMMUTER 3', '4_PM PM COMMUTER 4', '5_AM AM COMMUTER 5',
       '6_AM AM COMMUTER 6', '7_AM AM COMMUTER 7', '9_AM AM COMMUTER 9',
       'A ROUTE A', 'B ROUTE B', 'C ROUTE C', 'D ROUTE D', 'E ROUTE E',
       'F ROUTE F', 'L ROUTE L', 'M ROUTE M', 'R ROUTE R', 'S ROUTE S',
       'AM COMMUTER 8'], dtype=object)

#### Closer look at Santa Maria 'Rt 1. Transit Ctr to Preisker Park Via N. Broadway'

In [40]:
def display_map(gdf: gpd.GeoDataFrame, route: str):
    gdf = gdf.loc[gdf.recent_combined_name.str.contains(route)][
        ["geometry", "route_id"]
    ]
    display(gdf.explore("route_id"))
    return

In [41]:
display_map(
    routes,
    "Rt 1. Transit Ctr to Preisker Park Via N. Broadway",
)

In [42]:
inspect_recent_rows(
    df,
    "Rt 1. Transit Ctr to Preisker Park Via N. Broadway",
)

,service_date,recent_combined_name,portfolio_organization_name,schedule_gtfs_dataset_key,route_long_name,route_short_name,route_id
108478,2025-01-15,1B__Rt 1. Transit Ctr to Preisker Park Via N. Broadway,City of Santa Maria,73105f2d1cabc8170ab066d96863c5d5,Rt 1. Transit Ctr to Preisker Park Via N. Broadway,1B,1B
108479,2025-01-15,1__Rt 1. Transit Ctr to Preisker Park Via N. Broadway,City of Santa Maria,73105f2d1cabc8170ab066d96863c5d5,Rt 1. Transit Ctr to Preisker Park Via N. Broadway,1,1
111091,2025-02-12,1B__Rt 1. Transit Ctr to Preisker Park Via N. Broadway,City of Santa Maria,73105f2d1cabc8170ab066d96863c5d5,Rt 1. Transit Ctr to Preisker Park Via N. Broadway,1B,1B
111092,2025-02-12,1__Rt 1. Transit Ctr to Preisker Park Via N. Broadway,City of Santa Maria,73105f2d1cabc8170ab066d96863c5d5,Rt 1. Transit Ctr to Preisker Park Via N. Broadway,1,1
113614,2025-03-12,1B__Rt 1. Transit Ctr to Preisker Park Via N. Broadway,City of Santa Maria,73105f2d1cabc8170ab066d96863c5d5,Rt 1. Transit Ctr to Preisker Park Via N. Broadway,1B,1B
113615,2025-03-12,1__Rt 1. Transit Ctr to Preisker Park Via N. Broadway,City of Santa Maria,73105f2d1cabc8170ab066d96863c5d5,Rt 1. Transit Ctr to Preisker Park Via N. Broadway,1,1
116238,2025-04-16,1B__Rt 1. Transit Ctr to Preisker Park Via N. Broadway,City of Santa Maria,73105f2d1cabc8170ab066d96863c5d5,Rt 1. Transit Ctr to Preisker Park Via N. Broadway,1B,1B
116239,2025-04-16,1__Rt 1. Transit Ctr to Preisker Park Via N. Broadway,City of Santa Maria,73105f2d1cabc8170ab066d96863c5d5,Rt 1. Transit Ctr to Preisker Park Via N. Broadway,1,1
118655,2025-05-14,1B__Rt 1. Transit Ctr to Preisker Park Via N. Broadway,City of Santa Maria,73105f2d1cabc8170ab066d96863c5d5,Rt 1. Transit Ctr to Preisker Park Via N. Broadway,1B,1B
118656,2025-05-14,1__Rt 1. Transit Ctr to Preisker Park Via N. Broadway,City of Santa Maria,73105f2d1cabc8170ab066d96863c5d5,Rt 1. Transit Ctr to Preisker Park Via N. Broadway,1,1


#### Closer look at Roseville '1_AM AM COMMUTER 1'
* There are two separate routes that show up in the map.

In [43]:
display_map(
    routes,
    "1_AM AM COMMUTER 1",
)

In [44]:
inspect_recent_rows(df, "AM COMMUTER 1")

,service_date,recent_combined_name,portfolio_organization_name,schedule_gtfs_dataset_key,route_long_name,route_short_name,route_id
118695,2025-05-14,1_AM__AM COMMUTER 1,City of Roseville,3f420af2aea56d3ba5a1b10f488f0575,AM COMMUTER 1,1_AM,1_AM
118704,2025-05-14,10_PM__PM COMMUTER 10,City of Roseville,3f420af2aea56d3ba5a1b10f488f0575,AM COMMUTER 10,10_AM,10_AM


#### Closer look at Roseville '9_AM__AM COMMUTER 9'
* There are two separate routes that show up in the map.
* recent_combined_name was __AM COMMUTER 9 	 then this route disappeared for 5 months between Dec to May 2025 and popped up under a different name.

In [45]:
display_map(
    routes,
    "9_AM AM COMMUTER 9",
)

In [46]:
inspect_recent_rows(df, "AM COMMUTER 9")

,service_date,recent_combined_name,portfolio_organization_name,schedule_gtfs_dataset_key,route_long_name,route_short_name,route_id
81335,2024-08-14,__AM COMMUTER 9,City of Roseville,13ff8c918cc62f49169d93a04864d8e7,AM COMMUTER 9,,9_AM
83799,2024-09-18,__AM COMMUTER 9,City of Roseville,fb746afc72ff40405cfefa6d23ab58a0,AM COMMUTER 9,,9_AM
86453,2024-10-14,__AM COMMUTER 9,City of Roseville,fb746afc72ff40405cfefa6d23ab58a0,AM COMMUTER 9,,9_AM
89639,2024-10-15,__AM COMMUTER 9,City of Roseville,fb746afc72ff40405cfefa6d23ab58a0,AM COMMUTER 9,,9_AM
92233,2024-10-16,__AM COMMUTER 9,City of Roseville,fb746afc72ff40405cfefa6d23ab58a0,AM COMMUTER 9,,9_AM
94822,2024-10-17,__AM COMMUTER 9,City of Roseville,fb746afc72ff40405cfefa6d23ab58a0,AM COMMUTER 9,,9_AM
96777,2024-10-18,__AM COMMUTER 9,City of Roseville,fb746afc72ff40405cfefa6d23ab58a0,AM COMMUTER 9,,9_AM
102497,2024-11-13,__AM COMMUTER 9,City of Roseville,fb746afc72ff40405cfefa6d23ab58a0,AM COMMUTER 9,,9_AM
105164,2024-12-11,__AM COMMUTER 9,City of Roseville,fb746afc72ff40405cfefa6d23ab58a0,AM COMMUTER 9,,9_AM
118701,2025-05-14,9_AM__AM COMMUTER 9,City of Roseville,3f420af2aea56d3ba5a1b10f488f0575,AM COMMUTER 9,9_AM,9_AM


### Mountain View C - AM__Charleston, Garcia & San Antonio (Counterclockwise loop)/'C - AM Charleston, Garcia & San Antonio (Counterclockwise loop)',
* 2 different route_id for the same route, just one for am/one for pm

In [47]:
routes.loc[
    routes.recent_combined_name
    == "C - AM Charleston, Garcia & San Antonio (Counterclockwise loop)"
][["geometry", "route_id"]].explore("route_id")

In [48]:
inspect_recent_rows(df, "Charleston, Garcia & San Antonio")

,service_date,recent_combined_name,portfolio_organization_name,schedule_gtfs_dataset_key,route_long_name,route_short_name,route_id
106989,2025-01-15,"C - AM__Charleston, Garcia & San Antonio (Counterclockwise loop)",Mountain View Transportation Management Association,6869ab0f62d858fcd5d070e651609224,"Charleston, Garcia & San Antonio (Counterclockwise loop)",C - AM,C - AM
106988,2025-01-15,"C - AM__Charleston, Garcia & San Antonio (Counterclockwise loop)",Mountain View Transportation Management Association,6869ab0f62d858fcd5d070e651609224,"Charleston, Garcia & San Antonio (Counterclockwise loop)",C - PM,C - PM
109664,2025-02-12,"C - AM__Charleston, Garcia & San Antonio (Counterclockwise loop)",Mountain View Transportation Management Association,6869ab0f62d858fcd5d070e651609224,"Charleston, Garcia & San Antonio (Counterclockwise loop)",C - PM,C - PM
109665,2025-02-12,"C - AM__Charleston, Garcia & San Antonio (Counterclockwise loop)",Mountain View Transportation Management Association,6869ab0f62d858fcd5d070e651609224,"Charleston, Garcia & San Antonio (Counterclockwise loop)",C - AM,C - AM
112226,2025-03-12,"C - AM__Charleston, Garcia & San Antonio (Counterclockwise loop)",Mountain View Transportation Management Association,6869ab0f62d858fcd5d070e651609224,"Charleston, Garcia & San Antonio (Counterclockwise loop)",C - AM,C - AM
112230,2025-03-12,"C - AM__Charleston, Garcia & San Antonio (Counterclockwise loop)",Mountain View Transportation Management Association,6869ab0f62d858fcd5d070e651609224,"Charleston, Garcia & San Antonio (Counterclockwise loop)",C - PM,C - PM
114685,2025-04-16,"C - AM__Charleston, Garcia & San Antonio (Counterclockwise loop)",Mountain View Transportation Management Association,6869ab0f62d858fcd5d070e651609224,"Charleston, Garcia & San Antonio (Counterclockwise loop)",C - AM,C - AM
114689,2025-04-16,"C - AM__Charleston, Garcia & San Antonio (Counterclockwise loop)",Mountain View Transportation Management Association,6869ab0f62d858fcd5d070e651609224,"Charleston, Garcia & San Antonio (Counterclockwise loop)",C - PM,C - PM
117130,2025-05-14,"C - AM__Charleston, Garcia & San Antonio (Counterclockwise loop)",Mountain View Transportation Management Association,6869ab0f62d858fcd5d070e651609224,"Charleston, Garcia & San Antonio (Counterclockwise loop)",C - AM,C - AM
117136,2025-05-14,"C - AM__Charleston, Garcia & San Antonio (Counterclockwise loop)",Mountain View Transportation Management Association,6869ab0f62d858fcd5d070e651609224,"Charleston, Garcia & San Antonio (Counterclockwise loop)",C - PM,C - PM


### Mountainview - 'A - PM Middlefield, Clyde & Whisman',

In [49]:
routes.loc[routes.recent_combined_name == "A - PM Middlefield, Clyde & Whisman"][
    ["geometry", "route_id"]
].explore("route_id")

In [50]:
inspect_recent_rows(df, "Middlefield, Clyde & Whisman")

,service_date,recent_combined_name,portfolio_organization_name,schedule_gtfs_dataset_key,route_long_name,route_short_name,route_id
90751,2024-10-16,"A - PM__Middlefield, Clyde & Whisman",Mountain View Transportation Management Association,6869ab0f62d858fcd5d070e651609224,"Middlefield, Clyde & Whisman",A - PM,A - PM
93356,2024-10-17,"A - PM__Middlefield, Clyde & Whisman",Mountain View Transportation Management Association,6869ab0f62d858fcd5d070e651609224,"Middlefield, Clyde & Whisman",A - PM,A - PM
96017,2024-10-18,"A - PM__Middlefield, Clyde & Whisman",Mountain View Transportation Management Association,6869ab0f62d858fcd5d070e651609224,"Middlefield, Clyde & Whisman",A - PM,A - PM
101774,2024-11-13,"A - PM__Middlefield, Clyde & Whisman",Mountain View Transportation Management Association,6869ab0f62d858fcd5d070e651609224,"Middlefield, Clyde & Whisman",A - PM,A - PM
104442,2024-12-11,"A - PM__Middlefield, Clyde & Whisman",Mountain View Transportation Management Association,6869ab0f62d858fcd5d070e651609224,"Middlefield, Clyde & Whisman",A - PM,A - PM
106987,2025-01-15,"A - PM__Middlefield, Clyde & Whisman",Mountain View Transportation Management Association,6869ab0f62d858fcd5d070e651609224,"Middlefield, Clyde & Whisman",A - PM,A - PM
109662,2025-02-12,"A - PM__Middlefield, Clyde & Whisman",Mountain View Transportation Management Association,6869ab0f62d858fcd5d070e651609224,"Middlefield, Clyde & Whisman",A - PM,A - PM
112223,2025-03-12,"A - PM__Middlefield, Clyde & Whisman",Mountain View Transportation Management Association,6869ab0f62d858fcd5d070e651609224,"Middlefield, Clyde & Whisman",A - PM,A - PM
114682,2025-04-16,"A - PM__Middlefield, Clyde & Whisman",Mountain View Transportation Management Association,6869ab0f62d858fcd5d070e651609224,"Middlefield, Clyde & Whisman",A - PM,A - PM
117129,2025-05-14,"A - PM__Middlefield, Clyde & Whisman",Mountain View Transportation Management Association,6869ab0f62d858fcd5d070e651609224,"Middlefield, Clyde & Whisman",A - PM,A - PM


## Check the feed keys for Palo Verde

In [60]:
crosswalk_df.loc[
    crosswalk_df.organization_name.str.contains(
        "Palo Verde Valley Transit Agency",
    )
]

,schedule_gtfs_dataset_key,name,organization_name,service_date
146,96680f72c0c59208e055c81be87ad4a9,Desert Roadrunner Schedule,Palo Verde Valley Transit Agency,2025-01-15
205,4383eb1cca04093020f1583f57f32d9b,Desert Roadrunner GMV Schedule,Palo Verde Valley Transit Agency,2025-01-15
147,96680f72c0c59208e055c81be87ad4a9,Desert Roadrunner Schedule,Palo Verde Valley Transit Agency,2025-02-12
206,4383eb1cca04093020f1583f57f32d9b,Desert Roadrunner GMV Schedule,Palo Verde Valley Transit Agency,2025-02-12
145,96680f72c0c59208e055c81be87ad4a9,Desert Roadrunner Schedule,Palo Verde Valley Transit Agency,2025-03-12
205,4383eb1cca04093020f1583f57f32d9b,Desert Roadrunner GMV Schedule,Palo Verde Valley Transit Agency,2025-03-12
145,96680f72c0c59208e055c81be87ad4a9,Desert Roadrunner Schedule,Palo Verde Valley Transit Agency,2025-04-16
211,4383eb1cca04093020f1583f57f32d9b,Desert Roadrunner GMV Schedule,Palo Verde Valley Transit Agency,2025-04-16
142,96680f72c0c59208e055c81be87ad4a9,Desert Roadrunner Schedule,Palo Verde Valley Transit Agency,2025-05-14
210,4383eb1cca04093020f1583f57f32d9b,Desert Roadrunner GMV Schedule,Palo Verde Valley Transit Agency,2025-05-14


In [66]:
df.columns

Index(['schedule_gtfs_dataset_key', 'name', 'route_id', 'route_long_name',
       'route_short_name', 'route_desc', 'service_date', 'combined_name',
       'route_id2', 'recent_combined_name', 'recent_route_id2',
       'portfolio_organization_name'],
      dtype='object')

In [67]:
desert_roadrunner_schd = df.loc[(df.name.str.contains("Desert Roadrunner Schedule"))]

In [70]:
desert_roadrunner_schd.shape

(180, 12)

In [68]:
desert_roadrunner_gmv_schd = df.loc[
    (df.name.str.contains("Desert Roadrunner GMV Schedule"))
]

In [69]:
desert_roadrunner_gmv_schd.shape

(162, 12)

In [75]:
set(list(desert_roadrunner_schd.recent_combined_name.unique()))

{'1__Blue Route',
 '2__Gold Route',
 '3__Red Route',
 '4__Green Route',
 '5__Silver Route',
 '6__Wellness Express'}

In [80]:
set(list(desert_roadrunner_schd.route_id.unique()))

{'1444', '358', '359', '360', '361', '6467'}

In [77]:
set(list(desert_roadrunner_gmv_schd.recent_combined_name.unique()))

{'1__Blue Route',
 '2__Gold Route',
 '3__Red Route',
 '4__Green Route',
 '5__Silver Route',
 '6__Wellness Express'}

In [81]:
set(list(desert_roadrunner_gmv_schd.route_id.unique()))

{'5964', '5965', '5966', '5967', '5969', '5970'}

In [78]:
desert_roadrunner_gmv_schd.service_date.max()

Timestamp('2025-05-14 00:00:00')

In [79]:
desert_roadrunner_schd.service_date.max()

Timestamp('2025-05-14 00:00:00')

In [83]:
desert_roadrunner_gmv_schd.loc[
    desert_roadrunner_gmv_schd.recent_combined_name == "6__Wellness Express"
][
    [
        "service_date",
        "recent_combined_name",
        "portfolio_organization_name",
        "schedule_gtfs_dataset_key",
        "route_long_name",
        "route_short_name",
        "route_id",
    ]
].sort_values(
    by=["service_date"]
)

,service_date,recent_combined_name,portfolio_organization_name,schedule_gtfs_dataset_key,route_long_name,route_short_name,route_id
26508,2023-09-13,6__Wellness Express,Palo Verde Valley Transit Agency,4383eb1cca04093020f1583f57f32d9b,Wellness Express,6,5970
31330,2023-10-09,6__Wellness Express,Palo Verde Valley Transit Agency,4383eb1cca04093020f1583f57f32d9b,Wellness Express,6,5970
34223,2023-10-11,6__Wellness Express,Palo Verde Valley Transit Agency,4383eb1cca04093020f1583f57f32d9b,Wellness Express,6,5970
39419,2023-10-13,6__Wellness Express,Palo Verde Valley Transit Agency,4383eb1cca04093020f1583f57f32d9b,Wellness Express,6,5970
45162,2023-11-15,6__Wellness Express,Palo Verde Valley Transit Agency,4383eb1cca04093020f1583f57f32d9b,Wellness Express,6,5970
47758,2023-12-13,6__Wellness Express,Palo Verde Valley Transit Agency,4383eb1cca04093020f1583f57f32d9b,Wellness Express,6,5970
50316,2024-01-17,6__Wellness Express,Palo Verde Valley Transit Agency,4383eb1cca04093020f1583f57f32d9b,Wellness Express,6,5970
54926,2024-02-14,6__Wellness Express,Palo Verde Valley Transit Agency,4383eb1cca04093020f1583f57f32d9b,Wellness Express,6,5970
55293,2024-03-13,6__Wellness Express,Palo Verde Valley Transit Agency,4383eb1cca04093020f1583f57f32d9b,Wellness Express,6,5970
57759,2024-04-15,6__Wellness Express,Palo Verde Valley Transit Agency,4383eb1cca04093020f1583f57f32d9b,Wellness Express,6,5970


In [84]:
desert_roadrunner_schd.loc[
    desert_roadrunner_schd.recent_combined_name == "6__Wellness Express"
][
    [
        "service_date",
        "recent_combined_name",
        "portfolio_organization_name",
        "schedule_gtfs_dataset_key",
        "route_long_name",
        "route_short_name",
        "route_id",
    ]
].sort_values(
    by=["service_date"]
)

,service_date,recent_combined_name,portfolio_organization_name,schedule_gtfs_dataset_key,route_long_name,route_short_name,route_id
2295,2023-03-15,6__Wellness Express,Palo Verde Valley Transit Agency,69c28a761e7b8186ccfab752f38df2dc,Wellness Express,6,6467
18999,2023-05-17,6__Wellness Express,Palo Verde Valley Transit Agency,69c28a761e7b8186ccfab752f38df2dc,Wellness Express,6,6467
21387,2023-06-14,6__Wellness Express,Palo Verde Valley Transit Agency,69c28a761e7b8186ccfab752f38df2dc,Wellness Express,6,6467
23810,2023-07-12,6__Wellness Express,Palo Verde Valley Transit Agency,69c28a761e7b8186ccfab752f38df2dc,Wellness Express,6,6467
28720,2023-09-13,6__Wellness Express,Palo Verde Valley Transit Agency,ac9384d5e25378d1898ca522070cef66,Wellness Express,6,6467
31456,2023-10-09,6__Wellness Express,Palo Verde Valley Transit Agency,ac9384d5e25378d1898ca522070cef66,Wellness Express,6,6467
36493,2023-10-11,6__Wellness Express,Palo Verde Valley Transit Agency,ac9384d5e25378d1898ca522070cef66,Wellness Express,6,6467
41647,2023-10-13,6__Wellness Express,Palo Verde Valley Transit Agency,ac9384d5e25378d1898ca522070cef66,Wellness Express,6,6467
47406,2023-11-15,6__Wellness Express,Palo Verde Valley Transit Agency,ac9384d5e25378d1898ca522070cef66,Wellness Express,6,6467
50175,2023-12-13,6__Wellness Express,Palo Verde Valley Transit Agency,ac9384d5e25378d1898ca522070cef66,Wellness Express,6,6467


In [85]:
routes.columns

Index(['shape_array_key', 'geometry', 'feed_key', 'schedule_gtfs_dataset_key',
       'route_id', 'direction_id', 'route_key', 'route_length',
       'route_length_miles', 'is_downtown_local', 'is_local', 'is_coverage',
       'is_rapid', 'is_express', 'is_rail', 'is_ferry',
       'organization_source_record_id', 'organization_name', 'service_date',
       'portfolio_organization_name', 'name', 'combined_name',
       'recent_combined_name', 'recent_route_id',
       'route_length_miles_percentile', 'percentile_group',
       'route_length_feet'],
      dtype='object')

In [88]:
routes.loc[
    routes.name.isin(["Desert Roadrunner GMV Schedule", "Desert Roadrunner Schedule"])
].recent_combined_name.unique()

array(['1 Blue Route', '2 Gold Route', '3 Red Route', '4 Green Route',
       '6 Wellness Express'], dtype=object)

In [90]:
for route in list(
    routes.loc[
        routes.name.isin(
            ["Desert Roadrunner GMV Schedule", "Desert Roadrunner Schedule"]
        )
    ].recent_combined_name.unique()
):
    display(route)
    display(
        routes.loc[routes.recent_combined_name == route][["name", "geometry"]].explore(
            "name"
        )
    )

'1 Blue Route'

'2 Gold Route'

'3 Red Route'

'4 Green Route'

'6 Wellness Express'

## Find  which routes have the same `recent_combined_name`,`route_long_name`, `route_short_name` but different operator + route_id info

In [95]:
agg2 = (
    with_pipe.groupby(
        [
            "service_date",
            "recent_combined_name",
            "portfolio_organization_name",
            "route_long_name",
            "route_short_name",
        ]
    )
    .agg({"route_id": "nunique"})
    .reset_index()
)

In [96]:
agg2_filtered = agg2.loc[(agg2.route_id > 1)]

In [97]:
agg2_filtered.shape

(329, 6)

In [98]:
agg2_filtered.sample()

,service_date,recent_combined_name,portfolio_organization_name,route_long_name,route_short_name,route_id
52353,2024-02-14,2__Gold Route,Palo Verde Valley Transit Agency,Gold Route,2,2


In [99]:
agg2_filtered.portfolio_organization_name.unique()

array(['City of Los Angeles', 'Amtrak',
       'Mission Bay Transportation Management Agency',
       'Palo Verde Valley Transit Agency',
       'San Luis Obispo Regional Transit Authority',
       'Los Angeles World Airports'], dtype=object)

In [167]:
agg2_filtered2 = agg2_filtered[
    ["service_date", "portfolio_organization_name", "recent_combined_name", "route_id"]
].drop_duplicates().sort_values(
    by=["portfolio_organization_name", "recent_combined_name", "service_date"]
)

In [168]:
agg2_filtered2.service_date.unique()

array(['2023-03-15', '2023-04-10', '2023-04-11', '2023-04-12',
       '2023-04-13', '2023-04-14', '2023-04-15', '2023-04-16',
       '2023-05-17', '2023-06-14', '2023-07-12', '2023-08-15',
       '2023-09-13', '2023-10-09', '2023-10-10', '2023-10-11',
       '2023-10-12', '2023-10-13', '2023-10-14', '2023-10-15',
       '2023-11-15', '2024-10-19', '2024-10-20', '2024-11-13',
       '2024-12-11', '2025-01-15', '2025-02-12', '2025-03-12',
       '2025-04-16', '2025-05-14', '2023-12-13', '2024-01-17',
       '2024-02-14', '2024-03-13', '2024-04-15', '2024-04-16',
       '2024-04-17', '2024-04-18', '2024-04-19', '2024-05-22',
       '2024-06-12', '2024-07-17', '2024-08-14', '2024-09-18',
       '2024-10-14', '2024-10-15', '2024-10-16', '2024-10-17',
       '2024-10-18', '2024-04-20'], dtype=object)

In [169]:
agg2_filtered2.loc[agg2_filtered2.service_date >= "2024-12-11"]

,service_date,portfolio_organization_name,recent_combined_name,route_id
104608,2024-12-11,Amtrak,__Amtrak Thruway Connecting Service,12
107201,2025-01-15,Amtrak,__Amtrak Thruway Connecting Service,12
109778,2025-02-12,Amtrak,__Amtrak Thruway Connecting Service,11
112282,2025-03-12,Amtrak,__Amtrak Thruway Connecting Service,12
114894,2025-04-16,Amtrak,__Amtrak Thruway Connecting Service,12
117298,2025-05-14,Amtrak,__Amtrak Thruway Connecting Service,12
104650,2024-12-11,Amtrak,__Commuter Rail,2
107243,2025-01-15,Amtrak,__Commuter Rail,2
109819,2025-02-12,Amtrak,__Commuter Rail,2
112321,2025-03-12,Amtrak,__Commuter Rail,2


### City of Los Angeles CE438B

In [160]:
crosswalk_df.loc[
    crosswalk_df.organization_name.str.contains(
        "City of Los Angeles",
    )
]

,schedule_gtfs_dataset_key,name,organization_name,service_date
181,cc53a0dbf5df90e3009b9cb5d89d80ba,LA DOT Schedule,City of Los Angeles,2025-01-15
182,cc53a0dbf5df90e3009b9cb5d89d80ba,LA DOT Schedule,City of Los Angeles,2025-02-12
179,cc53a0dbf5df90e3009b9cb5d89d80ba,LA DOT Schedule,City of Los Angeles,2025-03-12
185,cc53a0dbf5df90e3009b9cb5d89d80ba,LA DOT Schedule,City of Los Angeles,2025-04-16
184,cc53a0dbf5df90e3009b9cb5d89d80ba,LA DOT Schedule,City of Los Angeles,2025-05-14


In [102]:
inspect_recent_rows(df, "CE438B")

,service_date,recent_combined_name,portfolio_organization_name,schedule_gtfs_dataset_key,route_long_name,route_short_name,route_id
108221,2025-01-15,438B__CE438B,City of Los Angeles,cc53a0dbf5df90e3009b9cb5d89d80ba,CE438B,438B,1458
108224,2025-01-15,438B__CE438B,City of Los Angeles,cc53a0dbf5df90e3009b9cb5d89d80ba,CE438B,438B,4278
110822,2025-02-12,438B__CE438B,City of Los Angeles,cc53a0dbf5df90e3009b9cb5d89d80ba,CE438B,438B,1458
110825,2025-02-12,438B__CE438B,City of Los Angeles,cc53a0dbf5df90e3009b9cb5d89d80ba,CE438B,438B,4278
113263,2025-03-12,438B__CE438B,City of Los Angeles,cc53a0dbf5df90e3009b9cb5d89d80ba,CE438B,438B,1458
113409,2025-03-12,438B__CE438B,City of Los Angeles,cc53a0dbf5df90e3009b9cb5d89d80ba,CE438B,438B,4278
115972,2025-04-16,438B__CE438B,City of Los Angeles,cc53a0dbf5df90e3009b9cb5d89d80ba,CE438B,438B,1458
115974,2025-04-16,438B__CE438B,City of Los Angeles,cc53a0dbf5df90e3009b9cb5d89d80ba,CE438B,438B,4278
118383,2025-05-14,438B__CE438B,City of Los Angeles,cc53a0dbf5df90e3009b9cb5d89d80ba,CE438B,438B,1458
118386,2025-05-14,438B__CE438B,City of Los Angeles,cc53a0dbf5df90e3009b9cb5d89d80ba,CE438B,438B,4278


In [103]:
display_map(routes, "CE438B")

### Amtrak __Amtrak Thruway Connecting Service
* Map isn't working b/c no geometry

In [105]:
inspect_recent_rows(df, "Amtrak Thruway Connecting Service")

,service_date,recent_combined_name,portfolio_organization_name,schedule_gtfs_dataset_key,route_long_name,route_short_name,route_id
117567,2025-05-14,__Amtrak Thruway Connecting Service,Amtrak,48e137bc977da88970393f629c18432c,Amtrak Thruway Connecting Service,,42933
117564,2025-05-14,__Amtrak Thruway Connecting Service,Amtrak,48e137bc977da88970393f629c18432c,Amtrak Thruway Connecting Service,,42920
117563,2025-05-14,__Amtrak Thruway Connecting Service,Amtrak,48e137bc977da88970393f629c18432c,Amtrak Thruway Connecting Service,,18650
117557,2025-05-14,__Amtrak Thruway Connecting Service,Amtrak,48e137bc977da88970393f629c18432c,Amtrak Thruway Connecting Service,,43
117560,2025-05-14,__Amtrak Thruway Connecting Service,Amtrak,48e137bc977da88970393f629c18432c,Amtrak Thruway Connecting Service,,42896
117555,2025-05-14,__Amtrak Thruway Connecting Service,Amtrak,48e137bc977da88970393f629c18432c,Amtrak Thruway Connecting Service,,42891
117553,2025-05-14,__Amtrak Thruway Connecting Service,Amtrak,48e137bc977da88970393f629c18432c,Amtrak Thruway Connecting Service,,42954
117571,2025-05-14,__Amtrak Thruway Connecting Service,Amtrak,48e137bc977da88970393f629c18432c,Amtrak Thruway Connecting Service,,42958
117562,2025-05-14,__Amtrak Thruway Connecting Service,Amtrak,48e137bc977da88970393f629c18432c,Amtrak Thruway Connecting Service,,42912
117574,2025-05-14,__Amtrak Thruway Connecting Service,Amtrak,48e137bc977da88970393f629c18432c,Amtrak Thruway Connecting Service,,42935


In [151]:
thruway_gdf = routes.loc[routes.recent_combined_name.str.contains("Thruway Connecting")]

In [161]:
crosswalk_df.loc[
    crosswalk_df.organization_name.str.contains(
        "Amtrak",
    )
]

,schedule_gtfs_dataset_key,name,organization_name,service_date
103,48e137bc977da88970393f629c18432c,Amtrak Schedule,Amtrak,2025-01-15
104,48e137bc977da88970393f629c18432c,Amtrak Schedule,Amtrak,2025-02-12
104,48e137bc977da88970393f629c18432c,Amtrak Schedule,Amtrak,2025-03-12
103,48e137bc977da88970393f629c18432c,Amtrak Schedule,Amtrak,2025-04-16
100,48e137bc977da88970393f629c18432c,Amtrak Schedule,Amtrak,2025-05-14


### Palo Verde Valley Transit Agency 2__Gold Route
* two different route_id corresponds with the same long_name, short_name, and recent_combined name. it looks the same on the map.

In [116]:
routes.loc[
    routes.portfolio_organization_name == "Palo Verde Valley Transit Agency",
].recent_combined_name.unique()

array(['1 Blue Route', '2 Gold Route', '3 Red Route', '4 Green Route',
       '6 Wellness Express'], dtype=object)

In [115]:
inspect_recent_rows(df, "Gold Route")

,service_date,recent_combined_name,portfolio_organization_name,schedule_gtfs_dataset_key,route_long_name,route_short_name,route_id
103412,2024-12-11,2__Gold Route,Palo Verde Valley Transit Agency,4383eb1cca04093020f1583f57f32d9b,Gold Route,2,5965
107725,2025-01-15,2__Gold Route,Palo Verde Valley Transit Agency,96680f72c0c59208e055c81be87ad4a9,Gold Route,2,361
108420,2025-01-15,2__Gold Route,Palo Verde Valley Transit Agency,4383eb1cca04093020f1583f57f32d9b,Gold Route,2,5965
110430,2025-02-12,2__Gold Route,Palo Verde Valley Transit Agency,96680f72c0c59208e055c81be87ad4a9,Gold Route,2,361
111020,2025-02-12,2__Gold Route,Palo Verde Valley Transit Agency,4383eb1cca04093020f1583f57f32d9b,Gold Route,2,5965
113547,2025-03-12,2__Gold Route,Palo Verde Valley Transit Agency,4383eb1cca04093020f1583f57f32d9b,Gold Route,2,5965
115419,2025-04-16,2__Gold Route,Palo Verde Valley Transit Agency,96680f72c0c59208e055c81be87ad4a9,Gold Route,2,361
116171,2025-04-16,2__Gold Route,Palo Verde Valley Transit Agency,4383eb1cca04093020f1583f57f32d9b,Gold Route,2,5965
117828,2025-05-14,2__Gold Route,Palo Verde Valley Transit Agency,96680f72c0c59208e055c81be87ad4a9,Gold Route,2,361
118585,2025-05-14,2__Gold Route,Palo Verde Valley Transit Agency,4383eb1cca04093020f1583f57f32d9b,Gold Route,2,5965


In [117]:
display_map(routes, "2 Gold Route")

### San Luis Obispo LT Laguna Tripper
* Slightly different route geographies

In [164]:
crosswalk_df.loc[
    crosswalk_df.organization_name.str.contains(
        "San Luis Obispo Regional",
    )
]

,schedule_gtfs_dataset_key,name,organization_name,service_date
210,7891c0d5e91c8dccf88536129dbac084,SLORTA Schedule,San Luis Obispo Regional Transit Authority,2025-01-15
210,7891c0d5e91c8dccf88536129dbac084,SLORTA Schedule,San Luis Obispo Regional Transit Authority,2025-02-12
209,7891c0d5e91c8dccf88536129dbac084,SLORTA Schedule,San Luis Obispo Regional Transit Authority,2025-03-12
216,7891c0d5e91c8dccf88536129dbac084,SLORTA Schedule,San Luis Obispo Regional Transit Authority,2025-04-16
215,7891c0d5e91c8dccf88536129dbac084,SLORTA Schedule,San Luis Obispo Regional Transit Authority,2025-05-14


In [120]:
inspect_recent_rows(df, "Laguna Tripper")

,service_date,recent_combined_name,portfolio_organization_name,schedule_gtfs_dataset_key,route_long_name,route_short_name,route_id
105959,2025-01-15,LT__Laguna Tripper,San Luis Obispo Regional Transit Authority,f4c3ea214214ee0d96f7646b3e9d69dc,Laguna Tripper,LT,11806
107376,2025-01-15,LT__Laguna Tripper,San Luis Obispo Regional Transit Authority,b475cf3b7ceaae9fc30b22cb8d3a1e33,Laguna Tripper,LT,11096761
108549,2025-02-12,LT__Laguna Tripper,San Luis Obispo Regional Transit Authority,f4c3ea214214ee0d96f7646b3e9d69dc,Laguna Tripper,LT,11806
110068,2025-02-12,LT__Laguna Tripper,San Luis Obispo Regional Transit Authority,b475cf3b7ceaae9fc30b22cb8d3a1e33,Laguna Tripper,LT,11096761
111147,2025-03-12,LT__Laguna Tripper,San Luis Obispo Regional Transit Authority,f4c3ea214214ee0d96f7646b3e9d69dc,Laguna Tripper,LT,11806
112552,2025-03-12,LT__Laguna Tripper,San Luis Obispo Regional Transit Authority,b475cf3b7ceaae9fc30b22cb8d3a1e33,Laguna Tripper,LT,11096761
115676,2025-04-16,LT__Laguna Tripper,San Luis Obispo Regional Transit Authority,f4c3ea214214ee0d96f7646b3e9d69dc,Laguna Tripper,LT,11806
115737,2025-04-16,LT__Laguna Tripper,San Luis Obispo Regional Transit Authority,b475cf3b7ceaae9fc30b22cb8d3a1e33,Laguna Tripper,LT,11096761
116309,2025-05-14,LT__Laguna Tripper,San Luis Obispo Regional Transit Authority,f4c3ea214214ee0d96f7646b3e9d69dc,Laguna Tripper,LT,11806
117489,2025-05-14,LT__Laguna Tripper,San Luis Obispo Regional Transit Authority,b475cf3b7ceaae9fc30b22cb8d3a1e33,Laguna Tripper,LT,11096761


In [119]:
display_map(routes, "LT Laguna Tripper")

### Mission Bay Transportation Management Agency CCA/Adobe__
* Looks fine, I don't know why it was flagged considering it only has one unique row for each value

In [121]:
routes.loc[
    routes.portfolio_organization_name
    == "Mission Bay Transportation Management Agency",
].recent_combined_name.unique()

array(['CCA/Adobe', 'East', 'TC Transbay-Caltrain', 'West'], dtype=object)

In [165]:
crosswalk_df.loc[
    crosswalk_df.organization_name.str.contains(
        "Mission Bay",
    )
]

,schedule_gtfs_dataset_key,name,organization_name,service_date
68,749380f1a9f225d9123762d83ea2f50d,Bay Area 511 Mission Bay Schedule,Mission Bay Transportation Management Agency,2025-01-15
70,749380f1a9f225d9123762d83ea2f50d,Bay Area 511 Mission Bay Schedule,Mission Bay Transportation Management Agency,2025-02-12
71,749380f1a9f225d9123762d83ea2f50d,Bay Area 511 Mission Bay Schedule,Mission Bay Transportation Management Agency,2025-03-12
69,749380f1a9f225d9123762d83ea2f50d,Bay Area 511 Mission Bay Schedule,Mission Bay Transportation Management Agency,2025-04-16
68,749380f1a9f225d9123762d83ea2f50d,Bay Area 511 Mission Bay Schedule,Mission Bay Transportation Management Agency,2025-05-14


In [122]:
display_map(routes, "CCA/Adobe")

In [133]:
agg2_filtered.recent_combined_name.unique()

array(['CE438B', '__Amtrak Thruway Connecting Service', '__Commuter Rail',
       'CCA/Adobe__', 'East__', 'TC__Transbay-Caltrain', 'West__',
       '1__Blue Route', '2__Gold Route', '3__Red Route', '4__Green Route',
       '6__Wellness Express', '5__Silver Route', 'LT__Laguna Tripper',
       '__Maple Leaf', 'LAX to US__FlyAway - LAX to Union Station',
       'LAX to VN__FlyAway - LAX to Van Nuys',
       'US to LAX__FlyAway - Union Station to LAX',
       'VN to LAX__FlyAway - Van Nuys to LAX'], dtype=object)

In [139]:
(
    df.loc[df.recent_combined_name == "CCA/Adobe__"][
        [
            "service_date",
            "recent_combined_name",
            "portfolio_organization_name",
            "schedule_gtfs_dataset_key",
            "route_long_name",
            "route_short_name",
            "route_id",
        ]
    ]
    .sort_values(by=["service_date"])
    .drop_duplicates()
    .tail(10)
)

,service_date,recent_combined_name,portfolio_organization_name,schedule_gtfs_dataset_key,route_long_name,route_short_name,route_id
90747,2024-10-16,CCA/Adobe__,Mission Bay Transportation Management Agency,749380f1a9f225d9123762d83ea2f50d,,CCA/Adobe,CCA/Adobe
93352,2024-10-17,CCA/Adobe__,Mission Bay Transportation Management Agency,749380f1a9f225d9123762d83ea2f50d,,CCA/Adobe,CCA/Adobe
96013,2024-10-18,CCA/Adobe__,Mission Bay Transportation Management Agency,749380f1a9f225d9123762d83ea2f50d,,CCA/Adobe,CCA/Adobe
101768,2024-11-13,CCA/Adobe__,Mission Bay Transportation Management Agency,749380f1a9f225d9123762d83ea2f50d,,CCA/Adobe,CCA/Adobe
104436,2024-12-11,CCA/Adobe__,Mission Bay Transportation Management Agency,749380f1a9f225d9123762d83ea2f50d,,CCA/Adobe,CCA/Adobe
106980,2025-01-15,CCA/Adobe__,Mission Bay Transportation Management Agency,749380f1a9f225d9123762d83ea2f50d,,CCA/Adobe,CCA/Adobe
109656,2025-02-12,CCA/Adobe__,Mission Bay Transportation Management Agency,749380f1a9f225d9123762d83ea2f50d,,CCA/Adobe,CCA/Adobe
112219,2025-03-12,CCA/Adobe__,Mission Bay Transportation Management Agency,749380f1a9f225d9123762d83ea2f50d,,CCA/Adobe,CCA/Adobe
114678,2025-04-16,CCA/Adobe__,Mission Bay Transportation Management Agency,749380f1a9f225d9123762d83ea2f50d,,CCA/Adobe,CCA/Adobe
117125,2025-05-14,CCA/Adobe__,Mission Bay Transportation Management Agency,749380f1a9f225d9123762d83ea2f50d,,CCA/Adobe,CCA/Adobe


## LA World Airports LAX to US__FlyAway - LAX to Union Station

In [172]:
(
    df.loc[df.recent_combined_name.str.contains("LAX to Union Station")][
        [
            "service_date",
            "recent_combined_name",
            "portfolio_organization_name",
            "schedule_gtfs_dataset_key",
            "route_long_name",
            "route_short_name",
            "route_id",
        ]
    ]
    .sort_values(by=["service_date"])
    .drop_duplicates()
    .tail(10)
)

,service_date,recent_combined_name,portfolio_organization_name,schedule_gtfs_dataset_key,route_long_name,route_short_name,route_id
106407,2025-01-15,LAX to US__FlyAway - LAX to Union Station,Los Angeles World Airports,4a3f5fe2e305cc696684d11226ba9878,FlyAway - LAX to Union Station,LAX to US,6
108289,2025-01-15,LAX to US__FlyAway - LAX to Union Station,Los Angeles World Airports,2f1c266fc20f9875777fb752af32a66e,FlyAway - LAX to Union Station,LAX to US,TL-6
109057,2025-02-12,LAX to US__FlyAway - LAX to Union Station,Los Angeles World Airports,4a3f5fe2e305cc696684d11226ba9878,FlyAway - LAX to Union Station,LAX to US,6
110890,2025-02-12,LAX to US__FlyAway - LAX to Union Station,Los Angeles World Airports,2f1c266fc20f9875777fb752af32a66e,FlyAway - LAX to Union Station,LAX to US,TL-6
111636,2025-03-12,LAX to US__FlyAway - LAX to Union Station,Los Angeles World Airports,4a3f5fe2e305cc696684d11226ba9878,FlyAway - LAX to Union Station,LAX to US,6
113415,2025-03-12,LAX to US__FlyAway - LAX to Union Station,Los Angeles World Airports,2f1c266fc20f9875777fb752af32a66e,FlyAway - LAX to Union Station,LAX to US,TL-6
114160,2025-04-16,LAX to US__FlyAway - LAX to Union Station,Los Angeles World Airports,73e3164043eb7312454dd03413a6b310,FlyAway - LAX to Union Station,LAX to US,6
116040,2025-04-16,LAX to US__FlyAway - LAX to Union Station,Los Angeles World Airports,2f1c266fc20f9875777fb752af32a66e,FlyAway - LAX to Union Station,LAX to US,TL-6
116607,2025-05-14,LAX to US__FlyAway - LAX to Union Station,Los Angeles World Airports,be0f451de12b510fe2af5d47fe3cd0e7,FlyAway - LAX to Union Station,LAX to US,6
118453,2025-05-14,LAX to US__FlyAway - LAX to Union Station,Los Angeles World Airports,2f1c266fc20f9875777fb752af32a66e,FlyAway - LAX to Union Station,LAX to US,TL-6


## Los Angeles World Airports VN to LAX__FlyAway - Van Nuys to LAX
* Two different route_ids for the same short_name, long_name, recent_combined_name for May 2025.

In [144]:
routes.loc[
    routes.portfolio_organization_name == "Los Angeles World Airports",
].recent_combined_name.unique()

array(['VN to LAX FlyAway - Van Nuys to LAX',
       'US to LAX FlyAway - Union Station to LAX',
       'Inter-terminal Connector/AC Route',
       'LAX to VN FlyAway - LAX to Van Nuys',
       'LAX to US FlyAway - LAX to Union Station', 'Metro Connector GL',
       'Metro C Line Shuttle M to Terminals',
       'Metro C Line Shuttle M to Metro Rail',
       'LAX FlyAway-US2LAX Union Station FlyAway to LAX',
       'LAX FlyAway-LAX2US FlyAway to Union Station/Downtown',
       'LAX FlyAway-VN2LAX Van Nuys FlyAway to LAX',
       'LAX FlyAway-LAX2VN FlyAway to Van Nuys', 'ADA Shuttle Route',
       'LAX Economy Parking'], dtype=object)

In [148]:
(
    df.loc[df.recent_combined_name.str.contains("Van Nuys")][
        [
            "service_date",
            "recent_combined_name",
            "portfolio_organization_name",
            "schedule_gtfs_dataset_key",
            "route_long_name",
            "route_short_name",
            "route_id",
        ]
    ]
    .sort_values(by=["service_date"])
    .drop_duplicates()
    .tail(10)
)

,service_date,recent_combined_name,portfolio_organization_name,schedule_gtfs_dataset_key,route_long_name,route_short_name,route_id
114157,2025-04-16,VN to LAX__FlyAway - Van Nuys to LAX,Los Angeles World Airports,73e3164043eb7312454dd03413a6b310,FlyAway - Van Nuys to LAX,VN to LAX,1
116005,2025-04-16,Van Nuys/Studio City Counterclockwise__DASH Van Nuys/Studio City Counterclockwise,City of Los Angeles,cc53a0dbf5df90e3009b9cb5d89d80ba,DASH Van Nuys/Studio City Counterclockwise,Van Nuys/Studio City Counterclockwise,800
118450,2025-05-14,VN to LAX__FlyAway - Van Nuys to LAX,Los Angeles World Airports,2f1c266fc20f9875777fb752af32a66e,FlyAway - Van Nuys to LAX,VN to LAX,TL-1
116604,2025-05-14,VN to LAX__FlyAway - Van Nuys to LAX,Los Angeles World Airports,be0f451de12b510fe2af5d47fe3cd0e7,FlyAway - Van Nuys to LAX,VN to LAX,1
116606,2025-05-14,LAX to VN__FlyAway - LAX to Van Nuys,Los Angeles World Airports,be0f451de12b510fe2af5d47fe3cd0e7,FlyAway - LAX to Van Nuys,LAX to VN,5
118415,2025-05-14,Van Nuys/Studio City Clockwise__DASH Van Nuys/Studio City Clockwise,City of Los Angeles,cc53a0dbf5df90e3009b9cb5d89d80ba,DASH Van Nuys/Studio City Clockwise,Van Nuys/Studio City Clockwise,799
118416,2025-05-14,Van Nuys/Studio City Counterclockwise__DASH Van Nuys/Studio City Counterclockwise,City of Los Angeles,cc53a0dbf5df90e3009b9cb5d89d80ba,DASH Van Nuys/Studio City Counterclockwise,Van Nuys/Studio City Counterclockwise,800
118433,2025-05-14,Panorama City/Van Nuys Clockwise__DASH Panorama City/Van Nuys Clockwise,City of Los Angeles,cc53a0dbf5df90e3009b9cb5d89d80ba,DASH Panorama City/Van Nuys Clockwise,Panorama City/Van Nuys Clockwise,801
118434,2025-05-14,Panorama City/Van Nuys Counterclockwise__DASH Panorama City/Van Nuys Counterclockwise,City of Los Angeles,cc53a0dbf5df90e3009b9cb5d89d80ba,DASH Panorama City/Van Nuys Counterclockwise,Panorama City/Van Nuys Counterclockwise,804
118452,2025-05-14,LAX to VN__FlyAway - LAX to Van Nuys,Los Angeles World Airports,2f1c266fc20f9875777fb752af32a66e,FlyAway - LAX to Van Nuys,LAX to VN,TL-5


In [146]:
display_map(routes, "VN to LAX FlyAway - Van Nuys to LAX")

In [166]:
crosswalk_df.loc[
    crosswalk_df.organization_name.str.contains(
        "Los Angeles World Airports",
    )
]

,schedule_gtfs_dataset_key,name,organization_name,service_date
29,4a3f5fe2e305cc696684d11226ba9878,LAX FlyAway Schedule,Los Angeles World Airports,2025-01-15
182,723210f3a6d61ee3936df401e18a5636,LAX Shuttles Schedule,Los Angeles World Airports,2025-01-15
183,2f1c266fc20f9875777fb752af32a66e,LAX Flyaway Bus Schedule,Los Angeles World Airports,2025-01-15
31,4a3f5fe2e305cc696684d11226ba9878,LAX FlyAway Schedule,Los Angeles World Airports,2025-02-12
183,723210f3a6d61ee3936df401e18a5636,LAX Shuttles Schedule,Los Angeles World Airports,2025-02-12
184,2f1c266fc20f9875777fb752af32a66e,LAX Flyaway Bus Schedule,Los Angeles World Airports,2025-02-12
32,4a3f5fe2e305cc696684d11226ba9878,LAX FlyAway Schedule,Los Angeles World Airports,2025-03-12
180,723210f3a6d61ee3936df401e18a5636,LAX Shuttles Schedule,Los Angeles World Airports,2025-03-12
181,2f1c266fc20f9875777fb752af32a66e,LAX Flyaway Bus Schedule,Los Angeles World Airports,2025-03-12
32,73e3164043eb7312454dd03413a6b310,LAX FlyAway Schedule,Los Angeles World Airports,2025-04-16
